In [ ]:
import os
import json
import pandas as pd

def ler_jsons_em_diretorio_com_lotes(diretorio, diretorio_raw, chunksize=1000):
    arquivos_json = [f for f in os.listdir(diretorio) if f.endswith('.json')]
    
    if not arquivos_json:
        print("🚨 Nenhum arquivo JSON encontrado no diretório.")
        return
    
    os.makedirs(diretorio_raw, exist_ok=True)
    
    lote = 1
    for arquivo in arquivos_json:
        caminho_arquivo = os.path.join(diretorio, arquivo)
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            dados = json.load(f)
            
            # ✅ Flatten do JSON
            df = pd.json_normalize(dados, record_path=['entry'], sep='_')
            total_registros = len(df)
            print(f"✅ Arquivo {arquivo} carregado com {total_registros} registros.")

            # ✅ Tratamento de dados (conversão de listas e valores misturados)
            for coluna in df.columns:
                if df[coluna].apply(lambda x: isinstance(x, list)).any():
                    # Converte listas para strings ou explode em várias linhas
                    df[coluna] = df[coluna].apply(
                        lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x
                    )
                
                # Converte valores misturados em strings
                df[coluna] = df[coluna].astype(str)

            # ✅ Divide em lotes e salva em Parquet
            for i in range(0, total_registros, chunksize):
                df_lote = df.iloc[i:i + chunksize].reset_index(drop=True)
                
                caminho_saida = os.path.join(diretorio_raw, f'lote_{lote:03d}.parquet')
                
                # 🔥 Salva cada lote em arquivo Parquet
                df_lote.to_parquet(caminho_saida, engine='pyarrow', index=False)
                print(f"✅ Lote {lote} salvo em: {caminho_saida}")

                lote += 1

# 🔥 Exemplo de uso:
diretorio = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\data'
diretorio_raw = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\raw'
chunksize = 10000  # ✅ Define o tamanho do lote

# ✅ Cria o diretório de saída, se não existir
os.makedirs(diretorio_raw, exist_ok=True)

# ✅ Executa o processo com controle de lote
ler_jsons_em_diretorio_com_lotes(diretorio, diretorio_raw, chunksize)

✅ Arquivo Aaron697_Brekke496_2fa15bc7-8866-461a-9000-f739e425860a.json carregado com 319 registros.
✅ Lote 1 salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\raw\lote_001.parquet
✅ Arquivo Aaron697_Stiedemann542_41166989-975d-4d17-b9de-17f94cb3eec1.json carregado com 556 registros.
✅ Lote 2 salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\raw\lote_002.parquet
✅ Arquivo Abby752_Kuvalis369_2b083021-e93f-4991-bf49-fd4f20060ef8.json carregado com 183 registros.
✅ Lote 3 salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\raw\lote_003.parquet
✅ Arquivo Abel832_Connelly992_29e51479-f742-4474-8f8e-d2607d5269f6.json carregado com 183 registros.
✅ Lote 4 salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\raw\lote_004.parquet
✅ Arquivo Abraham100_Heller342_262b819a-5193-404a-9787-b7f599358035.json carregado com 198 registros.
✅ Lote 5 salvo em: C:\Users\Leonardo\OneDrive\Documentos\

In [ ]:
#import glob

#arquivos = glob.glob(r'C:/Users/Leonardo/OneDrive/Documentos/Git/teste_engenheiro/eng_dados/raw/*.parquet')
#df = pd.concat([pd.read_parquet(f) for f in arquivos], ignore_index=True)

#for col in df.columns:
#    print(col)

#pip freeze > requirements.txt